# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`
import text: 

mongoimport --type json -d uk_food -c establishments --file /Users/pcn08/ucb/homework/nosql-challenge/Module_12_Files/Resources/establishments.json --drop --jsonArray

In [ ]:
#or mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

In [5]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
#On the mongosh terminal
show dbs

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
show collections

In [ ]:
# review the collections in our new database
show collections

In [ ]:
# review a document in the establishments collection


In [ ]:
db.establishments.find()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data


In [ ]:
{
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":true
}

In [ ]:
  insertedId: ObjectId("65bc4d90eab55afa0e2bb833")
}
uk_food> db.establishments.find({_id: ObjectId("65bc4d90eab55afa0e2bb833")})

In [ ]:
# Insert the new restaurant into the collection
db.establishments.insertOne({
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":true
})

#had to change True to true for it to work on Mongosh 

In [ ]:
# Check that the new restaurant was inserted
db.establishments.find({_id: ObjectId("65bc4d90eab55afa0e2bb833")})

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields


In [ ]:
db.establishments.find({BusinessType:'Restaurant/Cafe/Canteen'})

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID


In [ ]:
db.establishments.updateOne({_id: ObjectId ("65bc4d90eab55afa0e2bb833")}, {$set: {"BusinessTypeID": 1}})

In [ ]:
# Confirm that the new restaurant was updated
db.establishments.find({"BusinessName":"Penang Flavours","BusinessTypeID": 1})

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
db.establishments.find({"LocalAuthorityName":"Dover"})

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
db.establishments.deleteMany({"LocalAuthorityName":"Dover"})

In [ ]:
# Check if any remaining documents include Dover
db.establishments.find({"LocalAuthorityName":"Dover"})

In [ ]:
# Check that other documents remain with 'find_one'
db.establishments.findOne({"LocalAuthorityName": "Dover"})

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude


In [ ]:
db.establishments.updateMany({}, [ {'$set':{ "geocode.longitude" : {'$toInt': "$geocode.longitude"}}} ])

In [ ]:
db.establishments.updateMany({}, [
  { '$set': { "geocode.longitude": { '$toDouble': "$geocode.longitude" }, "geocode.latitude": { '$toDouble': "$geocode.latitude" } } }
])


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue


In [ ]:
db.establishments.update_many({}, [ {'$set': { "RatingValue" :{'$toInt': "$RatingValue"}
                                              } 
                                     } ]
                              )

In [ ]:
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]

db.establishments.updateMany(
    {
        "RatingValue": {
            "$exists": true,
            "$nin": non_ratings
        }
    },
    [
        {
            "$set": {
                "RatingValue": {
                    "$toInt": "$RatingValue"
                }
            }
        }
    ]
)


In [ ]:
# Check that the coordinates and rating value are now numbers


In [ ]:
db.establishments.find({
  "geocode.latitude": { "$type": "double" },
  "geocode.longitude": { "$type": "double" }
})

db.establishments.find({
  "RatingValue": { "$type": "int" }

Notes: I used the actvitities covered in class to help me figure out the correct queries, see unit xyz for example. For multiple queries, I found this website that was useful (see line xyz), https://developerslogblog.wordpress.com/2019/10/15/mongodb-how-to-filter-by-multiple-fields/ 